# ArjanCode Tips

## ⚙️ Functions

[Video Url](https://www.youtube.com/watch?v=yatgY4NpZXE)

### 1. Do 1 thing and do it well

In [2]:
from dataclasses import dataclass
import datetime

In [3]:
@dataclass
class Customer:
    name: str
    phone: str
    cc_number: str
    cc_exp_month: int
    cc_exp_year: int
    cc_valid: bool = False

In [4]:
## OLD VERSION (Before refactoring)

def validate_card(customer: Customer) -> bool:
    def digitits_of(number: str) -> list[int]:
        return [int(d) for d in number]
    
    digits = digitits_of(customer.cc_number)
    odd_digits = digits[-1::-2]
    even_digits = digits[-2::-2]
    checksum = 0
    checksum += sum(odd_digits)
    for digit in even_digits:
        checksum += sum(digitits_of(str(digit * 2)))

    customer.cc_valid = (
        checksum % 10 == 0
        and datetime(customer.cc_exp_year, customer.cc_exp_month, 1) > datetime.now()
    )
    return customer.cc_valid

In [5]:
def main() -> None:
    alice = Customer(
        name="alice",
        phone="2341",
        cc_number="124",
        cc_exp_month=1,
        cc_exp_year=2024,
    )
    is_valid = validate_card(alice)
    print(f"Is Alice's card valid? {is_valid}")
    print(alice)

main()

Is Alice's card valid? False
Customer(name='alice', phone='2341', cc_number='124', cc_exp_month=1, cc_exp_year=2024, cc_valid=False)


In [6]:
## NEW VERSION 
def luhn_checksum(card_number: str) -> bool:
    def digitits_of(number: str) -> list[int]:
        return [int(d) for d in number]
    
    digits = digitits_of(card_number)
    odd_digits = digits[-1::-2]
    even_digits = digits[-2::-2]
    checksum = 0
    checksum += sum(odd_digits)
    for digit in even_digits:
        checksum += sum(digitits_of(str(digit * 2)))
    return checksum % 10 == 0

def validate_card(customer: Customer) -> bool:
    customer.cc_valid = (
        luhn_checksum(customer.cc_number)
        and datetime(customer.cc_exp_year, customer.cc_exp_month, 1) > datetime.now()
    )
    return customer.cc_valid

### 2. Separate commands from queries

In [7]:
def validate_card(customer: Customer) -> bool:
    return (
        luhn_checksum(customer.cc_number)
        and datetime(customer.cc_exp_year, customer.cc_exp_month, 1) > datetime.now()
    )

def main() -> None:
    alice = Customer(
        name="alice",
        phone="2341",
        cc_number="124",
        cc_exp_month=1,
        cc_exp_year=2024,
    )
    alice.cc_valid = validate_card(alice)
    print(f"Is Alice's card valid? {alice.cc_valid}")
    print(alice)

main()

Is Alice's card valid? False
Customer(name='alice', phone='2341', cc_number='124', cc_exp_month=1, cc_exp_year=2024, cc_valid=False)


### 3. Only request information you actually need

In [8]:
# validate_card was requesting the full Customer, but was not required all the Customer parameters
def validate_card(number: int, exp_year: int, exp_month:int) -> bool:
    return (
        luhn_checksum(number)
        and datetime(exp_year, exp_month, 1) > datetime.now()
    )

def main() -> None:
    alice = Customer(
        name="alice",
        phone="2341",
        cc_number="124",
        cc_exp_month=1,
        cc_exp_year=2024,
    )
    alice.cc_valid = validate_card(
        number = alice.cc_number, 
        exp_month = alice.cc_exp_year, 
        exp_year = alice.cc_exp_month
    )
    
    print(f"Is Alice's card valid? {alice.cc_valid}")
    print(alice)

main()

Is Alice's card valid? False
Customer(name='alice', phone='2341', cc_number='124', cc_exp_month=1, cc_exp_year=2024, cc_valid=False)


### 4. Keep the number of parameters minimal

In [ ]:
from typing import Protocol

class CardInfo(Protocol)